# Cover Page

## STUDENT ID No: 210006819

## MODULE CODE: GG4257

## MODULE TITLE: Urban Analytics: A Toolkit for Sustainable Urban Development

## ASSIGNMENT: Independent Research Project

## DEGREE PROGRAMME: International Relations and Geography

## WORD COUNT: N/A

## DEADLINE DATE: 05/01/2025

In submitting this assignment I hereby confirm that:

I have read the University's statement on Good Academic Practice; that the following work is my own work; and that significant academic debts and borrowings have been properly acknowledged and referenced.

# Reproducing the Code

This section introduces how to replicate the code and how to get the required data. The code used to produce these maps can be found in a separate jupyter notebook called Code_Final_Assessment.ipynb. While it generally follows the structure outlined in the methodology, under each figure in the report, the code corresponding to its creation will be cited for ease of reproducibility and transparency.

The notebook of the completed report will be accessible in my Github Repo as "Report_Final_Assessment". Furthermore, the data needed to reproduce these maps will also be uploaded to a google drive folder. It will also be made available in a folder called "data" uploaded to this repository.

## [Google Drive Folder with Data](https://drive.google.com/drive/folders/1la5A6jrOylBtMjtsFkAid9vLym6hIm2G?usp=drive_link)
## [My Github Repo](https://github.com/nb22-coder/210006819_UA_IRP)
## <a href="Code_Final_Assessment.ipynb#-Reproducing-the-Code">Notebook with the code</a>

### Libraries for Upload

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
!pip install folium
import folium
import shapely.geometry
from shapely.geometry import MultiPolygon, Polygon
from shapely.geometry import Point
from shapely import wkt

# 1. Vulnerability Data
The data for displacement is sourced from the Indices of Multiple Deprivation dataset. The code below must combine the 2019 Indices for Multiple Deprivation with the geometries of the LSOAs (chosen granularity).

### 1.1 LSOA Data

In [ ]:
# IMD 2019 File
lsoa_geo = gpd.read_file("LSOA geometry/LSOA_2004_London_Low_Resolution.shp")
# I need to first find how many LSOAs are in London
lsoa_geo["LSOA_CODE"].count()

In [ ]:
lsoa_geo.head()

### 1.2 Indices of Multiple Deprivation Data 

In [ ]:
imd =pd.read_csv("Vulnerability Index/cdrc_imd_data.csv")

#Show column headings
imd.head()

# I need to first find how the LSOAs look in this file
imd["ls11cd"].count()

In [ ]:
imd.info()

### 1.3 Joining for Vulnerability Index

In [ ]:
#Ok so there is a discrepancy between the 'ls11cd' and the 'LSOA_CODE' but i need to initiate the merge nonetheless

# Here I check if the LSOA codes are strings (important for merging later)
lsoa_geo['LSOA_CODE'] = lsoa_geo['LSOA_CODE'].astype(str)
imd['ls11cd'] = imd['ls11cd'].astype(str)

# Now I initiate merge between the two so I can get the geo_data on the indices of multiple deprivation 
imd_merged = lsoa_geo.merge(imd, how='inner', left_on='LSOA_CODE', right_on='ls11cd')

print(imd_merged.shape)
print(imd_merged.head())


In [ ]:
#Dropping unecessary columns found on the merge
print(imd_merged.columns.tolist())

columns_to_drop = ['MSOA_CODE', 'MSOA_NAME', 'STWARDCODE', 'STWARDNAME','ls11cd', 'la19nm', 'england_imd_rank', 'england_imd_decile']

imd_merged = imd_merged.drop(columns=columns_to_drop)

In [ ]:
print(imd_merged.columns.tolist())

In [ ]:
# Here I make sure there are not Nans
print(imd_merged.isnull().values.any())

Now that I have conducted this merge I need to mirror the index so that 1= lowest vulnerability to displacement, to 10=highest vulnerability to displacement.

In [ ]:
imd_merged['london_imd_decile'] = 11 - imd_merged['london_imd_decile']

# 2. Visualising Vulnerability Index

## 2.1 Descriptive Stats

In [ ]:
# Group by borough and describe IMD decile
borough_dstats = imd_merged.groupby('LA_NAME')['london_imd_decile'].describe()

borough_dstats = borough_dstats.sort_values('mean', ascending=False)

# Display the result
print(borough_dstats)


## 2.2 Modelling Vulnerability Index
This measure is much more straightforward for understanding vulnerability to displacement because it is a summative measure of multiple types of deprivation.

In [ ]:
imd_merged.head()

In [ ]:
# Filter most and least vulnerable LSOAs
most_vulnerable = imd_merged[imd_merged['london_imd_decile'] == 10]
least_vulnerable = imd_merged[imd_merged['london_imd_decile'] == 1]

# Display the top rows of each group
print("Most vulnerable LSOAs:")
print(most_vulnerable[['LSOA_CODE', 'LA_NAME', 'london_imd_decile']].head(10))

print("\nLeast vulnerable LSOAs:")
print(least_vulnerable[['LSOA_CODE', 'LA_NAME', 'london_imd_decile']].head(10))


## 2.3 Chloropleth of Vulnerability Index

In [ ]:
# Here I am plotting total vulnerability, using IMD as a proxy for vulnerability to displacement and here I am using a chloropleth map to visualise this
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
imd_merged.plot(column='london_imd_decile',
             cmap='Reds',
             legend=True,
             edgecolor='black',
             linewidth=0.2,
             ax=ax)

ax.set_title('London Vulnerability (IMD) Deciles by LSOA', fontsize=16)
ax.axis('off')
plt.show()
# Darker red (higher score) = more deprived

In [ ]:
# Here I am segmenting the vulnerability decile by borough to represent the boroughs most vulnerable to displacement due to their relative deprivation.
la_imd = imd_merged.groupby('LA_NAME')['london_imd_decile'].mean().sort_values()

# Bar plot
la_imd.plot(kind='barh', figsize=(10, 12), color='red')
plt.title('Average Vulnerability (IMD) Decile by London Borough')
plt.xlabel('Average Decile (Higher = More Vulnerable)')
plt.ylabel('London Borough')
plt.grid(True)
plt.show()


## 2.4 Supplementary Information: Barriers to Housing and Services Domain

While vulnerability is constituted by a myriad of factors, barriers to housing and services are essential for showing how housing informs the wider dimensions of vulnerability within areas of London. 


In [ ]:
# Here we can see boroughs like Richmond upon Thames, Kingston upon Thames, and Bromley have the lowest vulnerability scores, versus Tower Hamlets, Newham, Hackney, and Barking and Dagenham are all listed as the most vulnerable to displacement
hsv_dstats = imd_merged.groupby('LA_NAME')['london_imd_decile'].describe()

hsv_dstats = hsv_dstats.sort_values('mean')

print(hsv_dstats)


## 2.5 Chloropleth of Housing and Services Vulnerability

In [ ]:
# Here I am subsetting to visualise housing and services vulnerability
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
imd_merged.plot(column='barriers_london_decile',
             cmap='coolwarm',
             legend=True,
             edgecolor='black',
             linewidth=0.2,
             ax=ax)

ax.set_title('London Housing and Services Vulnerability Deciles by LSOA', fontsize=16)
ax.axis('off')
plt.show()
# Red means more barriers

In [ ]:
# Here are the average housing and services vulnerability (ex. housing affordability, homelessness, household overcrowding)
la_imd = imd_merged.groupby('LA_NAME')['barriers_london_decile'].mean().sort_values()

# This bar plot shows the subsetted vulnerability just for housing and services access
la_imd.plot(kind='barh', figsize=(10, 12), color='dodgerblue')
plt.title('Average Housing and Services Vulnerability Decile by London Borough')
plt.xlabel('Average Decile (Higher = More Vulnerable)')
plt.ylabel('London Borough')
plt.grid(True)
plt.show()


## 2.6 Interactive Model of Displacement Vulnerability Index

In [ ]:
# Here I'm setting the orientation of the base map
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Now I'm projecting the imd_merged geodataframe onto the base map to show the 
folium.Choropleth(
    geo_data=imd_merged,
    data=imd_merged,
    columns=['LSOA_CODE', 'london_imd_decile'],
    key_on='feature.properties.LSOA_CODE',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='London IMD Decile',
    highlight=True,
).add_to(m)

# Here is where I make the folium map clockable so you can explore the visualisation in greater detail
style_function = lambda x: {
    'fillColor': 'transparent',
    'color': 'black',
    'weight': 0.1,
    'fillOpacity': 0
}

popup_fields = ['LSOA_CODE', 'LA_NAME', 'london_imd_decile', 'london_imd_rank', 'barriers_london_decile', 'barriers_london_rank']

popup = folium.GeoJson(
    imd_merged,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=popup_fields,
        aliases=[
            'LSOA Code:',
            'Borough:',
            'Vulnerability (IMD) Decile:',
            'Vulnerability (IMD) Rank:',
            'Barriers to Housing Vulnerability Decile:',
            'Barriers to Housing Vulnerability Rank:'
        ],
        sticky=True
    )
)

popup.add_to(m)

# This is to work with the layer control
folium.LayerControl().add_to(m)

# I can export this file now 
m.save('london_imd_clickable_map.html')
m


# 3. House Price Change (Neighborhood Change Factor 1)
LSOA-level house price change and average weekly income change will be combined to reflect the socioeconomic shifts occurring in neighborhoods that are vulnerable to displacement from gentrification. 
## 3.1 Cleaning and Loading House Price Change Data (Neighborhood Change Factor 1)
This data is sourced from the Land Registry but it only exists on a borough level not LSOA so we will need to simulate this data.

In [ ]:
hp_index_LSOA =pd.read_csv("Neighborhood Change/land-registry-house-prices-LSOA.csv")

# Cleaning the index file
hp_index_LSOA.head()


In [ ]:
print(hp_index_LSOA.columns.tolist())

In [ ]:
# Dropping all empty columns and rows with Nan
hp_index_LSOA.isnull().sum()
columns_to_drop_hp = [ 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57']

hp_index_LSOA = hp_index_LSOA.drop(columns=columns_to_drop_hp)

In [ ]:
# There is an awkward space between the titles of some of the variables so we need to remove these

In [ ]:
print(hp_index_LSOA.columns.tolist())

In [ ]:
hp_index_LSOA.isnull().sum().sum()

In [ ]:
# Now all of them are removed and the dataset is properly cleaned
hp_index_LSOA = hp_index_LSOA.dropna()
hp_index_LSOA.isnull().sum().sum()

In [ ]:
hp_index_LSOA.head()

## 3.2 Projecting LSOAs onto House Price Data

In [ ]:
#Ok so there is a discrepancy between the 'ls11cd' and the 'LSOA_CODE' but i need to initiate the merge nonetheless

# Checking if the LSOA codes are strings so we can merge
lsoa_geo['LSOA_CODE'] = lsoa_geo['LSOA_CODE'].astype(str)
hp_index_LSOA['code'] = hp_index_LSOA['code'].astype(str)

# Initiating the merge between the geometry from the LSOA data and the house price index data
hp_merged = lsoa_geo.merge(hp_index_LSOA, how='inner', left_on='LSOA_CODE', right_on='code')

print(hp_merged.shape)
print(hp_merged.head())


In [ ]:
# REVISIT HERE - EDITS
# Nearest Neighbor Interpolation for multiple columns
def nearest_neighbor_interpolate(hp_merged, target_columns):
    # Ensure the target columns are numeric and handle missing values
    hp_merged[target_columns] = hp_merged[target_columns].apply(pd.to_numeric, errors='coerce')

    # Create a mask of where the data is missing for each column
    missing_masks = {col: hp_merged[col].isna() for col in target_columns}

    # Extract coordinates for each geometry (using centroids)
    coords = np.array([geom.representative_point().coords[0] for geom in hp_merged.geometry])
    
    # Loop through rows with missing values
    for idx, row in hp_merged.iterrows():
        for col in target_columns:
            if pd.isna(row[col]):  # If the value is missing
                # Find the nearest neighbor based on the distance between centroids
                distances = np.linalg.norm(coords - coords[idx], axis=1)  # Euclidean distance between centroids
                nearest_idx = np.argmin(distances[~missing_masks[col]])  # Find the closest non-missing value for the column
                hp_merged.at[idx, col] = hp_merged.iloc[nearest_idx][col]
    
    return hp_merged

# Example usage
# Assuming `hp_merged` has already been loaded

# Perform Nearest Neighbor Interpolation on the 'YE_dec_2017' and 'YE_dec_2007' columns
hp_merged = nearest_neighbor_interpolate(hp_merged, target_columns=['YE_ mar_2017', 'YE_ dec_2007'])


In [ ]:
# Choosing a start and an end date to model percentage growth
start_col = 'YE_ mar_2007'
end_col = 'YE_ dec_2017'

In [ ]:
# To create the equation I need to make sure the variables can be manipulated as numbers
hp_merged[start_col] = pd.to_numeric(hp_merged[start_col], errors='coerce')
hp_merged[end_col] = pd.to_numeric(hp_merged[end_col], errors='coerce')

In [ ]:
# Check missing values
print(hp_index_LSOA[[start_col, end_col]].isnull().sum())

In [ ]:
print(hp_index_LSOA[[start_col, end_col]].isnull().sum())

In [ ]:
hp_merged.head()

In [ ]:
hp_merged.isnull().values.any()

In [ ]:
check_columns = ['YE_ mar_2007','YE_ dec_2017']
if hp_merged[check_columns].isnull().values.any():
    print("There are missing values in the specified columns.")


    non_missing = hp_merged.dropna(subset=check_columns)
    missing = hp_merged[hp_merged[check_columns].isnull().any(axis=1)]

else:
    print("No missing values in the specified columns.")
    non_missing = hp_merged 
    missing = hp_merged.iloc[0:0]

print(f"Non-missing rows: {len(non_missing)}")
print(f"Missing rows: {len(missing)}")

In [ ]:
# Check if there are any missing values in the target columns
missing_values_after = hp_merged[['YE_ mar_2007', 'YE_ dec_2017']].isna().sum()

print("Missing values after interpolation:")
print(missing_values_after)


# 4. Solving for House Price AAGR

## 4.1 Calculating House Price AAGR

In [ ]:
years = 2017 - 2007  # 10 years

hp_merged['price_aagr'] = ((hp_merged[end_col] / hp_merged[start_col]) ** (1/years) - 1) * 100


This data still needs to be normalized once I'm working with more values

## 4.2 Modelling House Price AAGR

In [ ]:
# Plotting AAGR distribution
plt.figure(figsize=(10,6))
sns.histplot(hp_merged['price_aagr'], bins=30, kde=True, color='teal')
plt.title('Distribution of Average Annual Growth Rate in House Prices (AAGR) Across LSOAs')
plt.xlabel('AAGR (%)')
plt.ylabel('Number of LSOAs')
plt.grid(True)
plt.show()


### 4.2.1 Bar Plot for House Price AAGR 

In [ ]:
# Grouping AAGR by the borough to model change in housing prices
borough_aagr = hp_merged.groupby('LA_NAME')['price_aagr'].mean().sort_values(ascending=False)

# Plotting
plt.figure(figsize=(10,12))
borough_aagr.plot(kind='barh', color='teal')
plt.title('Average Annual Growth Rate (AAGR) in Housing Prices by London Borough')
plt.xlabel('Average AAGR (%)')
plt.ylabel('Borough')
plt.grid(True)
plt.show()


### 4.2.2 Scatterplot for House Price AAGR

In [ ]:
# Using scatterplots to model outliers
plt.figure(figsize=(12,8))
sns.boxplot(x='LA_NAME', y='price_aagr', data=hp_merged)
plt.xticks(rotation=90)
plt.title('Housing Prices AAGR Distribution Across LSOAs within Boroughs')
plt.xlabel('Borough')
plt.ylabel('AAGR (%)')
plt.grid(True)
plt.show()


### 4.2.3 Line Chart of House Price AAGR

In [ ]:
# REVISIT HERE - EDITS
# Modelling three subplots of the house price AAGR distribution
# Filter only YE_* columns 
price_columns = [col for col in hp_merged.columns if col.startswith('YE_')]

# Remove rows with ':' or non-numeric data
hp_merged = hp_merged[~hp_merged[price_columns].apply(lambda row: row.astype(str).str.contains(':').any(), axis=1)]
hp_merged[price_columns] = hp_merged[price_columns].apply(pd.to_numeric, errors='coerce')
hp_merged = hp_merged.dropna(subset=price_columns)

# Group by LA_NAME and calculate average prices
hp_df_grouped = hp_merged.groupby('LA_NAME')[price_columns].mean()

# Transpose for plotting (so columns are boroughs, rows are years)
hp_df_grouped = hp_df_grouped.T
hp_df_grouped.index = [col.replace('YE_', '') for col in hp_df_grouped.index]  # clean up x-axis labels

# Rank boroughs by latest available year
latest_prices = hp_df_grouped.iloc[-1]  # last row = most recent quarter
top10_boroughs = latest_prices.sort_values(ascending=False).head(10).index
bottom10_boroughs = latest_prices.sort_values(ascending=True).head(10).index

# Prepare color palettes
palette_top = sns.color_palette("husl", 10)
palette_bottom = sns.color_palette("husl", 10)

# Plot
fig, axs = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

# Top 10
for i, borough in enumerate(top10_boroughs):
    axs[0].plot(hp_df_grouped.index, hp_df_grouped[borough], label=borough, color=palette_top[i])
axs[0].set_title("Top 10 Boroughs by Most Recent Average Housing Price")
axs[0].set_ylabel("Average Price (£)")
axs[1].set_xlabel("Year/Quarter")
axs[0].legend(loc='upper left')

# Bottom 10
for i, borough in enumerate(bottom10_boroughs):
    axs[1].plot(hp_df_grouped.index, hp_df_grouped[borough], label=borough, color=palette_bottom[i])
axs[1].set_title("Bottom 10 Boroughs by Most Recent Average Housing Price")
axs[1].set_xlabel("Year/Quarter")
axs[1].set_ylabel("Average Price (£)")
axs[1].legend(loc='upper left')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## 4.3 Interactive Model of Neighborhood Change Factor 1

In [ ]:
# Base map centered on London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Adding a House Price AAGR Choropleth
folium.Choropleth(
    geo_data=hp_merged,
    data=hp_merged,
    columns=['LSOA_CODE', 'price_aagr'],
    key_on='feature.properties.LSOA_CODE',
    fill_color='viridis_r', 
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Annual House Price Growth Rate (%)',
    highlight=True
).add_to(m)

# Creating a popup
folium.GeoJson(
    hp_merged,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.2,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['LSOA_NAME', 'price_aagr'],
        aliases=['LSOA:', 'Annual Growth Rate (%)'],
        localize=True
    )
).add_to(m)

# Saving the map as an html
m.save('lsoa_aagr_map.html')
m


# 5. Weekly Income Change (Neighborhood Change Factor 2)

## 5.1 Cleaning and Uploading Weekly Earnings Change

The average weekly pay between the years 2007-17 is reflected in borough-level data. For this dataset we must project LSOA-level data for proper visualisation of neighborhood change as caused by increasing average weekly income.

In [ ]:
# Loading the cleaned data
earnings_con = pd.read_csv('Neighborhood Change/earnings-residence-borough.csv')

# Checking the structure
print(earnings_con.head())

In [ ]:
imd_merged.head()

In [ ]:
# Stripping any spaces or notations that will confuse the running of the code
imd_merged['LA_NAME'] = imd_merged['LA_NAME'].str.strip()
earnings_con['area'] = earnings_con['area'].str.strip()

# Merging the weekly average income figures on the vulnerability index
merged_df = imd_merged.merge(earnings_con, left_on='LA_NAME', right_on='area', how='left')

print(merged_df.columns.tolist())

In [ ]:
merged_df.head()

## 5.2 Projecting LSOAs onto Weekly Earnings Data

In [ ]:
# Projecting weekly income per LSOA for each year
for year in range(2007, 2018):
    pay_col = f'pay_{year}'
    est_col = f'estimated_income_{year}'
    
# Just in case the column is missing I have it so it will skip over it and not force the projection
    if pay_col in merged_df.columns:
        merged_df[est_col] = merged_df[pay_col] * merged_df['london_imd_decile']

# Retaining the relevant columns
income_cols = [col for col in merged_df.columns if col.startswith('estimated_income_')]
output_cols = ['LSOA_CODE', 'LA_NAME', 'geometry','london_imd_rank', 'london_imd_decile'] + income_cols
merged_df[output_cols].to_file('lsoa_estimated_weekly_income_2007_2017.geojson', driver='GeoJSON')

print("Projected LSOA weekly pay for 2007–2017 saved to 'lsoa_estimated_weekly_income_2007_2017.geojson'")




In [ ]:
geo_earnings = gpd.read_file('lsoa_estimated_weekly_income_2007_2017.geojson')
print(geo_earnings.columns.tolist())

In [ ]:
# Define number of years between periods
# years is already defined 

# Make sure earnings columns are numeric
geo_earnings['estimated_income_2011'] = pd.to_numeric(geo_earnings['estimated_income_2011'], errors='coerce')
geo_earnings['estimated_income_2017'] = pd.to_numeric(geo_earnings['estimated_income_2017'], errors='coerce')

## 5.3 Calculating Average Weekly Income AAGR

In [ ]:
# Calculating AAGR in average weekly incomes
geo_earnings['earnings_aagr'] = ((geo_earnings['estimated_income_2017'] / geo_earnings['estimated_income_2011']) ** (1/years) - 1) * 100


In [ ]:
# Now we can see here how the projected incomes are reflected in their AAGR growth rate with these random LSOAs
geo_earnings[['LSOA_CODE', 'estimated_income_2011', 'estimated_income_2017', 'earnings_aagr']].sample(10)

## 5.3 Modelling Estimated AAGR in London

In [ ]:
# Here I am checking for valid geometries to ensure that this can be modelled geospatially
earnings_gdf = gpd.GeoDataFrame(geo_earnings, geometry='geometry')
earnings_gdf.head()

In [ ]:
earnings_gdf.crs

In [ ]:
earnings_gdf.set_crs(epsg=4326, inplace=True)


In [ ]:
earnings_gdf['geometry'] = hp_merged.geometry

In [ ]:
earnings_gdf.head()

In [ ]:
# Here I am checking for valid geometries to ensure that this can be modelled geospatially
earnings_gdf = gpd.GeoDataFrame(geo_earnings, geometry='geometry')

# Transforming the CRS to EPSG: 27700 
earnings_gdf = earnings_gdf.set_crs(epsg=4326, allow_override=True)

earnings_gdf = earnings_gdf.to_crs(epsg=27700) 

earnings_gdf.head()

In [ ]:
earnings_gdf = earnings_gdf[earnings_gdf.is_valid]

In [ ]:
print(earnings_gdf.geom_type.value_counts())
# Dtype is not correct so we need to designate the geometries

In [ ]:
earnings_gdf['geometry'] = earnings_gdf.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
earnings_gdf = gpd.GeoDataFrame(earnings_gdf, geometry='geometry', crs="EPSG:4326")

In [ ]:
def ensure_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

earnings_gdf['geometry'] = earnings_gdf['geometry'].apply(ensure_multipolygon)


In [ ]:
# Here I am subsetting to visualise housing and services vulnerability
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
earnings_gdf.plot(column='earnings_aagr',
             cmap='coolwarm',
             legend=True,
             edgecolor='black',
             linewidth=0.2,
             ax=ax)

ax.set_title('London Housing and Services Vulnerability Deciles by LSOA', fontsize=16)
ax.axis('off')
plt.show()
# Red means more barriers

In [ ]:
print(earnings_gdf.geometry.is_empty.sum())   # How many are empty?
print(earnings_gdf.geometry.notnull().sum())  # How many are non-null?


In [ ]:
earnings_gdf = earnings_gdf[earnings_gdf.geometry.notnull()]
earnings_gdf = earnings_gdf[~earnings_gdf.geometry.is_empty]
earnings_gdf = earnings_gdf[earnings_gdf.is_valid]

In [ ]:
earnings_gdf.crs

In [ ]:
earnings_gdf.set_crs(epsg=27700, inplace=True)  # Only if not already set

In [ ]:
# Plotting the choropleth map
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
earnings_gdf.plot(column="earnings_aagr", ax=ax, legend=True,
         legend_kwds={'label': "Average Annual Earnings Growth Rate (%)",
                      'orientation': "horizontal"})
plt.title("Earnings AAGR Across London")
plt.show()

In [ ]:
print(earnings_gdf.head())
print(earnings_gdf.crs)


In [ ]:
# Plotting AAGR distribution to see how we need to calculate the composite vulnerability score
plt.figure(figsize=(10,6))
sns.histplot(earnings_gdf['earnings_aagr'], bins=30, kde=True, color='purple')
plt.title('Distribution of Average Annual Growth Rate (AAGR) for Weekly Income Across LSOAs')
plt.xlabel('AAGR (%)')
plt.ylabel('Number of LSOAs')
plt.grid(True)
plt.show()

### 5.3.2 Bar Plot for Weekly Income AAGR 

In [ ]:
# Usec groupby function to sort the AAGR by London borough
borough_aagr = earnings_gdf.groupby('LA_NAME')['earnings_aagr'].mean().sort_values(ascending=False)

# Plot
plt.figure(figsize=(10,12))
borough_aagr.plot(kind='barh', color='purple')
plt.title('Average Annual Growth Rate (AAGR) in Weekly Income by London Borough')
plt.xlabel('Average AAGR (%)')
plt.ylabel('Borough')
plt.grid(True)
plt.show()


### 5.3.3 Line Chart of Weekly Income AAGR

In [ ]:
# Using an if and for function to isolate the columns that have to do with average weekly earnings
income_columns = [col for col in geo_earnings.columns if col.startswith('estimated_income_20')]


# Producing a copy so the cleaned segment doesnt interfere with existing geo_earnings
earnings_cleaned = geo_earnings.copy()

# Instructing the system to ignore the columns that contain colons to denote a lack of input
earnings_cleaned = earnings_cleaned[~earnings_cleaned[income_columns].apply(lambda row: row.astype(str).str.contains(':').any(), axis=1)]

# Now we're converting to a float64 so it can be adapted to a line chart
earnings_cleaned[income_columns] = earnings_cleaned[income_columns].apply(pd.to_numeric, errors='coerce')
earnings_cleaned = earnings_cleaned.dropna(subset=income_columns)

# Here is the grouping by boroughs
earnings_grouped = earnings_cleaned.groupby('LA_NAME')[income_columns].mean()

# Finding the mean of each borough
earnings_grouped = geo_earnings.groupby('LA_NAME')[income_columns].mean()

# Transpose for plotting (so columns are boroughs, rows are years)
earnings_grouped = earnings_grouped.T
earnings_grouped.index = [col.replace('estimated_income_', '') for col in earnings_grouped.index]


# Plotting
plt.figure(figsize=(14, 8))
for borough in earnings_grouped.columns:
    plt.plot(earnings_grouped.index, earnings_grouped[borough], label=borough)

plt.title("Average Projected Weekly Earnings by Borough Over Time")
plt.xlabel("Year/Quarter")
plt.ylabel("Average Weekly Income (£)")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Sorting for the income columns
income_columns = [col for col in geo_earnings.columns if col.startswith('estimated_income_20')]

# Cleaning the data
earnings_cleaned = geo_earnings.copy()
earnings_cleaned = earnings_cleaned[~earnings_cleaned[income_columns].apply(lambda row: row.astype(str).str.contains(':').any(), axis=1)]
earnings_cleaned[income_columns] = earnings_cleaned[income_columns].apply(pd.to_numeric, errors='coerce')
earnings_cleaned = earnings_cleaned.dropna(subset=income_columns)

# Step 3: Group by borough and calculate average earnings
earnings_grouped = earnings_cleaned.groupby('LA_NAME')[income_columns].mean()

# Step 4: Transpose for plotting
earnings_grouped = earnings_grouped.T
earnings_grouped.index = [col.replace('estimated_income_', '') for col in earnings_grouped.index]

# Step 5: Rank boroughs by latest year's earnings
latest_earnings = earnings_grouped.iloc[-1]
top10 = latest_earnings.sort_values(ascending=False).head(10).index
bottom10 = latest_earnings.sort_values().head(10).index

# Step 6: Plot
fig, axs = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

# Top 10
palette_top = sns.color_palette("husl", 10)
for i, borough in enumerate(top10):
    axs[0].plot(earnings_grouped.index, earnings_grouped[borough], label=borough, color=palette_top[i])
axs[0].set_title("Top 10 Boroughs by Latest Average Earnings")
axs[0].set_ylabel("Average Earnings (£)")
axs[0].legend(loc='upper left')

# Bottom 10
palette_bottom = sns.color_palette("husl", 10)
for i, borough in enumerate(bottom10):
    axs[1].plot(earnings_grouped.index, earnings_grouped[borough], label=borough, color=palette_bottom[i])
axs[1].set_title("Bottom 10 Boroughs by Latest Average Earnings")
axs[1].set_xlabel("Year")
axs[1].set_ylabel("Average Earnings (£)")
axs[1].legend(loc='upper left')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## 5.4 Interactive Model of Weekly Income AAGR

In [ ]:
# Base map centered on London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Adding an Earnings AAGR Choropleth
folium.Choropleth(
    geo_data=earnings_gdf,
    data=earnings_gdf,
    columns=['LSOA_CODE', 'earnings_aagr'],
    key_on='feature.properties.LSOA_CODE',
    fill_color='viridis_r', 
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Annual Earnings Growth Rate (%)',
    highlight=True
).add_to(m)

# Creating a popup
folium.GeoJson(
    earnings_gdf,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.2,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['LA_NAME', 'earnings_aagr'],
        aliases=['LSOA:', 'Annual Growth Rate (%)'],
        localize=True
    )
).add_to(m)

# Saving the map as an html
m.save('earnings_aagr_map.html')
m

# 6. Computing a Neighborhood Change Indicator

## 6.1 Combining into the Neighborhood Change indicator

Now is where I need to normalise indicators using (min-max or z-score), I am electing to normalise using min-max because the earnings AAGR is not normally distributed

In [ ]:
# First I need to merge my two indicators into a single dataframe, for ease of processing I can cut out quite a few columns to make it as straightforward as possible
neighborhood_gdf = earnings_gdf.merge(hp_merged, on='LSOA_CODE', how='inner')
neighborhood_gdf.head(5)

In [ ]:
print(neighborhood_gdf.columns.tolist())

In [ ]:
# Subsetting the attributes we need, we dont need the total for now.
keep_cols= [
    'LSOA_CODE',
     'LA_NAME_x',
    'geometry_x',
     'london_imd_rank',
     'london_imd_decile',
     'earnings_aagr',
     'price_aagr'
]

neighborhood_gdf = neighborhood_gdf[keep_cols]

In [ ]:
neighborhood_gdf.head()

In [ ]:
# Scaling both variables from 0 to 10
scaler = MinMaxScaler(feature_range=(0, 10))
scaled = scaler.fit_transform(neighborhood_gdf[['earnings_aagr', 'price_aagr']])

neighborhood_gdf['earnings_change_scaled'] = scaled[:, 0]
neighborhood_gdf['price_change_scaled'] = scaled[:, 1]

# Creating the change index (equal weighting since both influence displacement)
neighborhood_gdf['nb_change_index'] = (
    0.5 * neighborhood_gdf['earnings_change_scaled'] +
    0.5 * neighborhood_gdf['price_change_scaled']
)
neighborhood_gdf.rename(columns={'london_imd_decile': 'vuln_index'}, inplace=True)

In [ ]:
neighborhood_gdf.head(10)

In [ ]:
# Remove rows where 'nb_change_index' is NaN
neighborhood_gdf = neighborhood_gdf.dropna(subset=['nb_change_index'])

print(neighborhood_gdf.isnull().values.any())


## 6.2 Visualising Neighborhood Change Index

In [ ]:
nb_gdf = gpd.GeoDataFrame(neighborhood_gdf, geometry='geometry_x')

# Assigning a CRS to ensure accurate projection
nb_gdf = nb_gdf.set_crs(epsg=27700, allow_override=True)


In [ ]:
# Ensure 'LSOA_CODE' is a string that can be manipulated
nb_gdf['LSOA_CODE'] = nb_gdf['LSOA_CODE'].astype(str)

# Creating the base map that is centered on London
nb_map = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Adding choropleth layer with gradient
folium.Choropleth(
    geo_data=nb_gdf,
    name='Neighborhood Change Index',
    data=nb_gdf,
    columns=['LSOA_CODE', 'nb_change_index'],
    key_on='feature.properties.LSOA_CODE',
    fill_color='viridis',  # or try 'PuBuGn', 'Viridis', 'OrRd'
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Neighborhood Change Index',
    highlight=True
).add_to(nb_map)

# Here I am adding a tooltip so you can hover over the individual LSOAs
tooltip = folium.GeoJson(
    nb_gdf,
    style_function=lambda x: {'fillOpacity': 0, 'weight': 0.3},
    tooltip=folium.GeoJsonTooltip(
        fields=['LSOA_CODE', 'LA_NAME_x', 'nb_change_index'],
        aliases=['LSOA Code:', 'Borough:', 'Change Index:'],
        sticky=True
    )
)
tooltip.add_to(nb_map)

# Here is where I can control
folium.LayerControl().add_to(nb_map)

# Saving the file
nb_map.save('london_nbchange_gradient.html')
nb_map


# 7. Composite Displacement Vulnerability Score

Now is the time to calculate a composite displacement vulnerability index using our two factors: 
- vulnerability index (e.g., based on IMD)
- neighborhood change index (e.g., based on AAGR of earnings and housing prices)

These should both now be on a 0–10 scale, with higher = more at risk/change.

Map the Overlap and Tensions
Key variables for spatial visualization:
- vulnerability_index
- neighborhood_change_index
- displacement_risk_scaled

Now we can map a bivariate chloropleth to show distribution of vulnerability and neighborhood change

Here composite vulnerability can be visualised using:

- High vulnerability + High change → Most at risk

- High vulnerability + Low change → Stable but vulnerable

- Low vulnerability + High change → Changing but resilient

- Low vulnerability + Low change → Low risk

## 7.1 Defining Composite Displacement Risk Score (CDRS)

In [ ]:
scaler = MinMaxScaler()

nb_gdf.rename(columns={'london_imd_decile': 'vuln_index'}, inplace=True)


nb_gdf[['vuln_scaled', 'change_scaled']] = scaler.fit_transform(nb_gdf[['vuln_index', 'nb_change_index']])


In [ ]:
nb_gdf['displacement_risk'] = (nb_gdf['vuln_scaled'] + nb_gdf['change_scaled']) / 2

In [ ]:
nb_gdf['displacement_risk_10'] = nb_gdf['displacement_risk'] * 10

In [ ]:
print(nb_gdf[['displacement_risk', 'vuln_scaled', 'change_scaled']].corr())


In [ ]:
# Creating the Overlap/Tension Categories
def categorize(val):
    if val < 0.33:
        return 'Low'
    elif val < 0.66:
        return 'Medium'
    else:
        return 'High'

nb_gdf['vuln_cat'] = nb_gdf['vuln_index'].apply(categorize)
nb_gdf['change_cat'] = nb_gdf['nb_change_index'].apply(categorize)

# Combining for a bivariate category where both can be visualised
nb_gdf['bivariate_class'] = nb_gdf['vuln_cat'] + '-' + nb_gdf['change_cat']


In [ ]:
nb_gdf['vuln_cat'] = pd.qcut(nb_gdf['vuln_scaled'], q=3, labels=['Low', 'Medium', 'High'])
nb_gdf['change_cat'] = pd.qcut(nb_gdf['change_scaled'], q=3, labels=['Low', 'Medium', 'High'])

# Combining to make a bivariate class
nb_gdf['bivariate_class'] = nb_gdf['vuln_cat'].astype(str) + '-' + nb_gdf['change_cat'].astype(str)


## 7.2 Descriptive Statistics for Combined Displacement Risk Score

In [ ]:
# Here I need to compute the Combined Displacement Risk Score
nb_gdf['displacement_risk'] = nb_gdf['vuln_index'] + nb_gdf['nb_change_index']

scaler = MinMaxScaler(feature_range=(0, 10))
nb_gdf['displacement_risk_scaled'] = scaler.fit_transform(nb_gdf[['displacement_risk']])


# Grouping by borough and describing displacement index
displaced_dstats = nb_gdf.groupby('LA_NAME_x')['displacement_risk_scaled'].describe()

displaced_dstats = displaced_dstats.sort_values('mean', ascending=False)

# Showing the result
print(displaced_dstats)

In [ ]:
nb_gdf['displacement_risk'].hist(bins=20)
plt.title('Displacement Index Distribution')
plt.xlabel('Index Value')
plt.ylabel('Frequency')
plt.show()

# Looking at the displacement index distribution
print(nb_gdf[['displacement_risk', 'vuln_index', 'nb_change_index']].corr())

Here we can tell that displacement_risk is highly correlated with vuln_index (0.91). Nb_change_index has low correlation with displacement_risk (0.22), likely because of smaller value ranges. Vuln_index and nb_change_index are weakly negatively correlated (-0.22) indicating that areas more vulnerable (increased deprivation) tend to show slightly less neighborhood change.


## 7.3 Designating Bivariate Composite Risk Scores

In [ ]:
nb_gdf['vuln_cat'] = pd.qcut(nb_gdf['vuln_scaled'], q=3, labels=['Low', 'Medium', 'High'])
nb_gdf['change_cat'] = pd.qcut(nb_gdf['change_scaled'], q=3, labels=['Low', 'Medium', 'High'])

# Combine to here to make a bivariate class
nb_gdf['bivariate_class'] = nb_gdf['vuln_cat'].astype(str) + '-' + nb_gdf['change_cat'].astype(str)



In [ ]:
print(nb_gdf['bivariate_class'].value_counts())


In [ ]:
nb_gdf['displacement_risk'].hist(bins=20)
plt.title('Displacement Index Distribution')
plt.xlabel('Index Value')
plt.ylabel('Frequency')
plt.show()

## 7.4 Spatial Visualisation of Composite Risk Score

In [ ]:
# Here is where I designate the colors of the overlap bivariate map, its not just as straightforward as a normal chloropleth because of the bivariate classification
bivariate_colors = {
    'Low-Low': '#e8e8e8',
    'Medium-Low': '#ace4e4',
    'High-Low': '#5ac8c8',
    'Low-Medium': '#dfb0d6',
    'Medium-Medium': '#a5add3',
    'High-Medium': '#5698b9',
    'Low-High': '#be64ac',
    'Medium-High': '#8c62aa',
    'High-High': '#3b4994',
}


To find these color classifications I consulted the [hexadecimal color codes](https://htmlcolorcodes.com/) and used their random color generator to designate these bivariate classes. It took some trial and error to find colors which showed the bivariate nature of the data.

In [ ]:
dis_m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Inputting the bivariate choloropleth
folium.GeoJson(
    nb_gdf,
    style_function=lambda feature: {
        'fillColor': bivariate_colors.get(feature['properties']['bivariate_class'], 'gray'),
        'color': 'black',
        'weight': 0.1,
        'fillOpacity': 0.8,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['LSOA_CODE', 'LA_NAME_x', 'bivariate_class'],
        aliases=['LSOA Code', 'Borough', 'Overlap Category'],
    )
).add_to(dis_m)

dis_m.save('london_overlap_tensions_map.html')
dis_m

# 8. Geodemographic Classification

## 8.1 Collating and Cleaning the Census Data

In [ ]:
# This code cell DOES NOT need to be run as there is already a file called merged_census_data.csv 
# I have included the code to show what needs to be done to combine the census layers
# Here the CENSUS data is merged to itself as its split into 6 separate distinctions


csv_directory = "census_raw_data"

# We need a list of all CSV files in the folder
csv_files = [file for file in os.listdir(csv_directory) if file.endswith(".csv")]

# An empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Loop through each CSV file
for csv_file in csv_files:
    csv_path = os.path.join(csv_directory, csv_file) # We create a consistent path
    df_csv = pd.read_csv(csv_path, low_memory=False) #read each file
    # Concatenate/Merge all columns, there is a pitfall here, you will get a duplicate oa_code from all csv files.
    merged_data = pd.concat([merged_data, df_csv], axis=1)

# Remove duplicate columns (keep the first occurrence)
merged_data = merged_data.loc[:, ~merged_data.columns.duplicated()]

# Save the merged dataset. You might want to do some pre-processing.
merged_data.to_csv("census_raw_data/merged_census_data.csv", index=False)
# Be aware of the mixted dtype you are importing we unfortunatly have to deal with that later.
# eventually you can avoid this to define the dtype on import method.

In [ ]:
print(merged_data.columns.tolist())

In [ ]:
# Here I define  the list of 32 London boroughs
london_boroughs = [
    'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Camden',
    'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham',
    'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington',
    'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham',
    'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark',
    'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster'
]

london_boroughs_clean = [b.lower().strip() for b in london_boroughs] #This line of code is stripping the columns to be sure they all match

# Here we've got to identify the column that contains the area names — assuming it's the first one
area_col = merged_data.columns[0]

# Add a helper column for matching
merged_data['_area_clean'] = merged_data[area_col].astype(str).str.lower().str.strip()

# Filtering for rows where the area name starts with a London borough
filtered_data = merged_data[merged_data['_area_clean'].apply(
    lambda x: any(x.startswith(b) for b in london_boroughs_clean)
)]

# Dropping the helper column that we used
filtered_data.drop(columns=['_area_clean'], inplace=True)

# Saving and storing the result for reproducibility
filtered_data.to_csv("census_raw_data/london_census_data.csv", index=False)


In [ ]:
nb_gdf.head()

In [ ]:
# Here we are making sure the LSOAs are stripped and sorted as strings 
filtered_data['2021 super output area - lower layer'] = filtered_data['2021 super output area - lower layer'].astype(str).str.strip()
nb_gdf['LSOA_NAME_x'] = nb_gdf['LSOA_NAME_x'].astype(str).str.strip()


In [ ]:

# Here is where we perform the merge between the
census_merged_gdf = nb_gdf.merge(
    filtered_data,
    left_on='LSOA_CODE',
    right_on='mnemonic',
    how='inner'  # Only keeping the rows that match the other dataset
)


In [ ]:
print("Rows with missing census data after merge:",
      census_merged_gdf['Total'].isna().sum()) # Here I am checking that all rows are filled and properly assigned


## 8.2 Selecting Desired Variables

In [ ]:
print(census_merged_gdf.columns.tolist())

In [ ]:
keep_cols = [
    'LSOA_CODE',
    'LSOA_NAME_x', 
    'LA_CODE_x', 
    'LA_NAME_x', 
    'geometry_x', 
    'london_imd_rank', 
    'vuln_index',  
    'earnings_aagr', 
    'price_aagr', 'earnings_change_scaled',
    'price_change_scaled', 
    'nb_change_index', 
    'displacement_risk', 
    'displacement_risk_scaled', 
    'vuln_cat', 
    'change_cat', 
    'bivariate_class', 
    'Total', 
    'Aged 4 years and under', 
    'Aged 5 to 9 years', 
    'Aged 10 to 15 years', 
    'Aged 16 to 19 years', 
    'Aged 20 to 24 years', 
    'Aged 25 to 34 years', 
    'Aged 35 to 49 years', 
    'Aged 50 to 64 years', 
    'Aged 65 to 74 years', 
    'Aged 75 to 84 years', 
    'Aged 85 years and over', 
    'Total: All usual residents aged 16 years and over', 
    'No qualifications', 'Level 1 and entry level qualifications', 'Level 2 qualifications', 'Apprenticeship', 'Level 3 qualifications', 'Level 4 qualifications or above', 'Other qualifications', 'Total: All usual residents', 'Asian, Asian British or Asian Welsh', 'Black, Black British, Black Welsh, Caribbean or African', 'Mixed or Multiple ethnic groups', 'White', 'Other ethnic group', 'One-person household', 'Single family household', 'Other household types', 'Born in the UK', '10 years or more', '5 years or more, but less than 10 years', '2 years or more, but less than 5 years', 'Less than 2 years', 'Total: All households', 'Owned', 'Shared ownership', 'Social rented', 'Private rented', 'Lives rent free', 'Owns with a mortgage or loan or shared ownership', 'Private rented or lives rent free', 'Total: All usual residents.1']

**SELECTED CENSUS FACTORS**

| Variable Group      | Columns                                                             | Significance                                                           |
|---------------------|------------------------------------------------------------------------|----------------------------------------------------------------------|
| **Age**             | Aged 25 to 34 years, Aged 65 to 74 years, Aged 75 to 84 years, Aged 85 years and over | Identifies young vs aging populations                              |
| **Tenure**          | Private rented, Social rented, Owned, Owns with a mortgage or loan or shared ownership, Lives rent free | Tenure security and economic status                                  |
| **Education**       | Level 4 qualifications or above, No qualifications                | Proxies for economic mobility and professional status                |
| **Ethnicity**       | White, Black, Black British, Black Welsh, Caribbean or African, Asian, Asian British or Asian Welsh | Can help interpret patterns in demographic distribution                                        |
| **Household Type**  | One-person household, Single family household                     | Useful to infer life stage or vulnerability                 |

In [ ]:
selected_features = [
    'Aged 25 to 34 years', 'Aged 65 to 74 years', 'Aged 75 to 84 years', 'Aged 85 years and over',
    'Private rented', 'Social rented', 'Owned', 'Owns with a mortgage or loan or shared ownership', 'Lives rent free',
    'Level 4 qualifications or above', 'No qualifications',
    'White', 'Black, Black British, Black Welsh, Caribbean or African', 'Asian, Asian British or Asian Welsh',
    'One-person household', 'Single family household',
    'Less than 2 years', '10 years or more'
]

X = census_merged_gdf[selected_features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


## 8.3 Examining Correlations of Census Variables

In [ ]:
# We used the scaler to put all these in percentages so now we can just run the correlation matrix
selected_features = [
    'Aged 25 to 34 years', 'Aged 65 to 74 years', 'Aged 75 to 84 years', 'Aged 85 years and over',
    'Private rented', 'Social rented', 'Owned', 'Owns with a mortgage or loan or shared ownership', 'Lives rent free',
    'Level 4 qualifications or above', 'No qualifications',
    'White', 'Black, Black British, Black Welsh, Caribbean or African', 'Asian, Asian British or Asian Welsh',
    'One-person household', 'Single family household',
    'Less than 2 years', '10 years or more'
]

# Extracting and cleaning empty columns
X = census_merged_gdf[selected_features].fillna(0)

# Computing a correlation matrix
corr_matrix = X.corr()

plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1, square=True)
plt.title("Correlation Matrix of Selected Census Features")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# This shows all the variables but is hard to interpret so we need to focus in on 

In [ ]:
# Computing the correlation matrix
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')

# Defining the threshold
threshold = 0.75
highly_correlated = (corr.abs() > threshold) & (corr.abs() < 1.0)

plt.figure(figsize=(10, 8))
sns.heatmap(highly_correlated, cmap='coolwarm', cbar=False, annot=True)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.title('Highly Correlated Variables')
plt.show()


In [ ]:
selected_features = [
    'Aged 25 to 34 years', 'Aged 65 to 74 years', 'Aged 75 to 84 years', 'Aged 85 years and over',
    'Private rented', 'Social rented', 'Owned', 'Owns with a mortgage or loan or shared ownership', 'Lives rent free',
    'Level 4 qualifications or above', 'No qualifications',
    'White', 'Black, Black British, Black Welsh, Caribbean or African', 'Asian, Asian British or Asian Welsh',
    'One-person household',
    'Less than 2 years', '10 years or more'
]

X = census_merged_gdf[selected_features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Now lets check again....
corr_2 = X.corr()
corr_2.style.background_gradient(cmap='coolwarm')

threshold = 0.8
highly_correlated_2 = (corr_2.abs() > threshold) & (corr_2.abs() < 1.0)

plt.figure(figsize=(10, 8))
sns.heatmap(highly_correlated_2, cmap='coolwarm', cbar=False, annot=True)

plt.title('New Highly Correlated Variables')
plt.show()

#Wahoo!!!, but before running the next part of the process, we also need to get ride of the NaN values. What a nightmare I know!.

## 8.4 Defining the Optimum Number of Clusters

In [ ]:
# Using this elbow graph to accurately determine the optimum number of clusters

def elbow(census_merged_data, n):
    kMeansVar = [KMeans(n_clusters=k).fit(census_merged_data.values) for k in range(1, n)] #making use of list comprehensions.
    centroids = [X.cluster_centers_ for X in kMeansVar]
    k_euclid = [cdist(census_merged_data.values, cent) for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclid]
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(census_merged_data.values)**2)/census_merged_data.values.shape[0]
    bss = tss - wcss
    plt.plot(bss)
    plt.show()
 
elbow(X,10)

In [ ]:
# KMeans with 3 clusters, after the validation with the Elbow method.
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
labels = kmeans.predict(X)
cluster_centres = kmeans.cluster_centers_

X['Cluster'] = kmeans.labels_

## 9. Evaluating and Interpreting Cluster Centers

**POINT VARIANCE**

In [ ]:
plt.figure(figsize=(12, 8))

kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X)

X['Cluster'] = clusters

scaler = StandardScaler()
stand_data_scaled = scaler.fit_transform(X)

# PCA analysis.
pca = PCA(n_components=2).fit(stand_data_scaled)
pca_result = pca.transform(stand_data_scaled)

#Percentage of variance explained by each of the selected components.
variance_ratio = pca.explained_variance_ratio_

# Creating a scatter plot
fig = px.scatter(x=pca_result[:, 0], y=pca_result[:, 1], color=clusters,
                 labels={'color': 'Cluster'},
                 #title='Cluster Plot against 1st 2 Principal Components',
                 opacity=0.7,
                 width=800, 
                 height=800)

plt.tight_layout()
fig.show()

print(f"These two components explain {(variance_ratio.sum()*100):.2f}% of the point variability.")

This is a high point variability, which we can see that there is less intermixing of the different cluster datapoints. This suggests that the K means clusters are fitting quite well.

**POINT VARIANCE WITH 2 PRINCIPAL COMPONENTS**

In [ ]:
# Here is a static figure with the point variability included in the x/y-axis label.
# So we can see what variability is provided by each component.

kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X)

X['Cluster'] = clusters

# Standardize the data for PCA
scaler = StandardScaler()
stand_data_scaled = scaler.fit_transform(X)

# PCA
pca = PCA(n_components=2).fit(stand_data_scaled)
pca_result = pca.transform(stand_data_scaled)

#Percentage of variance explained by each of the selected components.
variance_ratio = pca.explained_variance_ratio_

plt.figure(figsize=(10, 6))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=clusters, palette='viridis', s=50, alpha=0.7)
plt.title('Cluster Plot against 1st 2 Principal Components')
plt.xlabel(f'Principal Component 1 variation: {variance_ratio[0]*100:.2f}%')
plt.ylabel(f'Principal Component 2 variation: {variance_ratio[1]*100:.2f}%')
plt.legend(title='Clusters')
plt.show()

## 9.1 Interpreting the Cluster Centers

In [ ]:
# Assigning a cluster value to each OA 
kmeans = KMeans(n_clusters=3, random_state=42) # Based on the elbow method, I'm using k = 3 
clusters = kmeans.fit_predict(X)

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

# Get the cluster centers
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=X.columns)

cluster_centers.head(5)

### Cluster 0

In [ ]:
# Get the cluster center values for the first row
first_row_centers = cluster_centers.iloc[0, :]

# Get the number of features
num_features = len(first_row_centers)

# Define the angles for polar coordinates
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=False)

# Repeat the first value at the end to close the circle
theta = np.append(theta, theta[0])
first_row_centers = np.append(first_row_centers.values, first_row_centers.values[0])

# Create the polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

# Plot the cluster centers
ax.plot(theta, first_row_centers, linewidth=1, color='blue', marker='o', label='Centers')

# Plot the average line
mean_value = np.mean(first_row_centers)
ax.plot(theta, [mean_value] * len(theta), color='red', linestyle='--', label='Average')

# Set the tick labels and rotation
ax.set_xticks(theta[:-1])
ax.set_xticklabels(cluster_centers.columns, rotation=90, ha='center')

# Add legend
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()


### Cluster 1

In [ ]:
second_row_centers = cluster_centers.iloc[1, :]

# len of features
num_features = len(second_row_centers)

# polar coordinates
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, second_row_centers, linewidth=1, color='blue', marker='o', label='Centers')
# Add an extra red line at the 0.0 value
ax.plot(theta, np.zeros_like(second_row_centers), color='red', linestyle='--', label='Average')

ax.set_xticks(theta)
ax.set_xticklabels(cluster_centers.columns, rotation=90, ha='right')

plt.show()
#Ignore the cluster polar values, and focus in he census variables.

### Cluster 2

In [ ]:
# Get cluster 3 data
third_row_centers = cluster_centers.iloc[2, :]

# Number of features
num_features = len(third_row_centers)

# Angles for polar plot
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=False)
theta = np.append(theta, theta[0])  # Close the loop
third_row_centers = np.append(third_row_centers.values, third_row_centers.values[0])

# Compute real average
mean_value_2 = np.mean(third_row_centers)

# Create polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, third_row_centers, linewidth=1, color='blue', marker='o', label='Centres')
ax.plot(theta, [mean_value_2]*len(theta), color='red', linestyle='--', label='Average')

# Set axis ticks
ax.set_xticks(theta[:-1])
ax.set_xticklabels(cluster_centers.columns, rotation=45, ha='right')

# Add legend and show
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
plt.tight_layout()
plt.show()
